## Prediction

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel, T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer, DataCollatorForSeq2Seq
import json
import requests
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
# Load the test dataset
test_dataset = load_dataset('json', data_files='./data/test_TLQA.json')['train']

In [3]:
# Load the generative model and tokenizer
generative_model_name = './results-FlanT5-small'
generative_tokenizer = T5Tokenizer.from_pretrained(generative_model_name)
generative_model = T5ForConditionalGeneration.from_pretrained(generative_model_name)
generative_model.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [4]:
def preprocess_function(examples):
    inputs = examples['question']
    targets = ["; ".join(ans) for ans in examples['answers']]  # Join list of answers with a separator
    model_inputs = generative_tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    with generative_tokenizer.as_target_tokenizer():
        labels = generative_tokenizer(targets, max_length=512, truncation=True, padding="max_length")

    model_inputs['labels'] = labels['input_ids']
    model_inputs['question'] = inputs  # Keep the original question
    return model_inputs

In [5]:
# Tokenize the test dataset
tokenized_test = test_dataset.map(preprocess_function, batched=True)

In [6]:
# Load the retriever model
retriever_model_name = "facebook/contriever"
retriever_tokenizer = AutoTokenizer.from_pretrained(retriever_model_name)
retriever_model = AutoModel.from_pretrained(retriever_model_name)

In [7]:
import re

def get_wikidata_context(wikidata_id, type_id):
    sparql_url = "https://query.wikidata.org/sparql"
    query = f"""
    SELECT ?item ?itemLabel ?startTime ?endTime WHERE {{
      wd:{wikidata_id} p:{type_id} ?membership.
      ?membership ps:{type_id} ?item.
      OPTIONAL {{ ?membership pq:P580 ?startTime. }}
      OPTIONAL {{ ?membership pq:P582 ?endTime. }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    """
    headers = {'Accept': 'application/json'}
    response = requests.get(sparql_url, params={'query': query}, headers=headers)
    data = response.json()

    contexts = []
    if 'results' in data and 'bindings' in data['results']:
        for binding in data['results']['bindings']:
            item_label = binding['itemLabel']['value']
            start_time = binding.get('startTime', {}).get('value', 'unknown')
            end_time = binding.get('endTime', {}).get('value', 'unknown')
            if re.match(r'^\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z$', start_time) and (end_time == 'unknown' or re.match(r'^\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z$', end_time)):
                context = (item_label, start_time, end_time)
                contexts.append(context)
    return contexts

In [8]:
# Function to calculate temporal score
def calculate_temporal_score(query_time, doc_time, alpha_scale):
    qt = np.datetime64(query_time)
    dt = np.datetime64(doc_time)
    return alpha_scale / (np.abs((qt - dt).astype('timedelta64[D]').astype(int)) + 1)

In [9]:
# Function to normalize temporal scores
def normalize_temporal_score(temporal_scores):
    mean_temp = np.mean(temporal_scores)
    std_temp = np.std(temporal_scores)
    return [(score - mean_temp) / std_temp for score in temporal_scores]

# Function to combine semantic and temporal scores
def temp_rett_score(semantic_scores, temporal_scores, alpha_scale):
    temporal_scores = normalize_temporal_score(temporal_scores)
    combined_scores = [s + alpha_scale * t for s, t in zip(semantic_scores, temporal_scores)]
    return combined_scores

In [10]:
def rank_and_filter_documents(semantic_scores, temporal_scores, query_time, contexts, k):
    combined_scores = temp_rett_score(semantic_scores, temporal_scores, alpha_scale=1.0)  # Adjust alpha_scale as needed
    ranked_docs = sorted(zip(combined_scores, contexts), key=lambda x: x[0], reverse=True)
    print("Ranked Docs:", ranked_docs)  # Debugging statement
    filtered_docs = [doc for score, doc in ranked_docs if np.datetime64(doc[1]) <= np.datetime64(query_time)]
    return filtered_docs[:k]


In [11]:
# Function to generate answers with context using the generative model
def generate_answers_with_context(question, context):
    combined_input = question + " " + context
    inputs = generative_tokenizer(combined_input, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    outputs = generative_model.generate(
        inputs['input_ids'],
        max_length=512,
        num_beams=5,
        early_stopping=True,
        repetition_penalty=2.5,
        length_penalty=1.0,
        no_repeat_ngram_size=2,
    )
    return generative_tokenizer.decode(outputs[0], skip_special_tokens=True)

In [12]:
import re
# Function to parse years from the question text
def parse_years_from_question(question):
    years = re.findall(r'\b\d{4}\b', question)
    if len(years) == 2:
        start_year, end_year = years
        return start_year, end_year
    return None, None

In [13]:
# Function to retrieve embeddings
def get_embeddings(texts, tokenizer, model):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings


In [14]:
from sklearn.metrics.pairwise import cosine_similarity

# Function to compute cosine similarity
def compute_cosine_similarity(query_emb, doc_embs):
    return cosine_similarity([query_emb], doc_embs)[0]

In [15]:
import warnings
# Main prediction function
def main_prediction(test_dataset):
    predictions = []
    for example in test_dataset:
        question = example['question']
        start_year, end_year = parse_years_from_question(question)
        if not start_year or not end_year:
            predictions.append("No valid years found in question")
            continue
        
        query_time = f"{end_year}-01-01T00:00:00Z"  # Use end year as the query time
        wikidata_id = example['wikidata_ID']
        relation_type = example['type']
        
        # Retrieve context
        contexts = get_wikidata_context(wikidata_id, relation_type)
        print("Contexts:", contexts)
        if not contexts:
            predictions.append("No context found")
            continue
        
        # Get query and document embeddings
        query_emb = get_embeddings([question], retriever_tokenizer, retriever_model)[0]
        doc_texts = [ctx[0] for ctx in contexts]
        doc_embs = get_embeddings(doc_texts, retriever_tokenizer, retriever_model)
        
        # Compute semantic scores
        semantic_scores = compute_cosine_similarity(query_emb, doc_embs)
        
        # Extract times and calculate temporal scores
        valid_contexts = [(ctx[0], ctx[1], ctx[2]) for ctx in contexts if ctx[1] != 'unknown' and re.match(r'^\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z$', ctx[1])]
        temporal_scores = [calculate_temporal_score(query_time, dt[1], alpha_scale=1.0) for dt in valid_contexts]
        
        # Ensure alignment of temporal scores with document times
        aligned_doc_times = [dt for dt in valid_contexts if dt[1] != 'unknown']
        
        # Rank and filter documents
        topk_contexts = rank_and_filter_documents(semantic_scores, temporal_scores, query_time, aligned_doc_times, k=5)
        print("Topk Contexts:", topk_contexts)  # Debugging statement
        
        # Combine top-k contexts into a single string
        combined_context = "; ".join([f"{ctx[0]} ({ctx[1]}, {ctx[2]})" for ctx in topk_contexts])
        
        # Generate answer
        answer = generate_answers_with_context(question, combined_context)
        predictions.append(answer)
    
    return predictions

In [16]:
predictions = main_prediction(test_dataset)
for question, prediction in zip(test_dataset['question'], predictions):
    print(f"Question: {question}\nPrediction: {prediction}\n")

Contexts: [('Oldham Athletic A.F.C.', '2006-01-01T00:00:00Z', '2006-01-01T00:00:00Z'), ('Southend United F.C.', '2008-01-01T00:00:00Z', '2012-01-01T00:00:00Z'), ('Luton Town F.C.', '2007-01-01T00:00:00Z', '2008-01-01T00:00:00Z'), ('Chelsea F.C.', '2004-01-01T00:00:00Z', '2008-01-01T00:00:00Z'), ('Stevenage F.C.', '2012-01-01T00:00:00Z', '2013-01-01T00:00:00Z'), ('Crewe Alexandra F.C.', '2013-01-01T00:00:00Z', '2015-01-01T00:00:00Z'), ('Port Vale F.C.', '2015-01-01T00:00:00Z', 'unknown'), ('Wycombe Wanderers F.C.', '2006-01-01T00:00:00Z', '2007-01-01T00:00:00Z'), ('Southend United F.C.', '2008-01-01T00:00:00Z', '2008-01-01T00:00:00Z')]


C:\Users\Win10\AppData\Local\Temp\ipykernel_2308\3988018708.py:3: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  qt = np.datetime64(query_time)
C:\Users\Win10\AppData\Local\Temp\ipykernel_2308\3988018708.py:4: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  dt = np.datetime64(doc_time)
C:\Users\Win10\AppData\Local\Temp\ipykernel_2308\1794962655.py:5: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  filtered_docs = [doc for score, doc in ranked_docs if np.datetime64(doc[1]) <= np.datetime64(query_time)]


Ranked Docs: [(2.5637509400607055, ('Port Vale F.C.', '2015-01-01T00:00:00Z', 'unknown')), (1.2256862651420015, ('Crewe Alexandra F.C.', '2013-01-01T00:00:00Z', '2015-01-01T00:00:00Z')), (0.8545167465612206, ('Stevenage F.C.', '2012-01-01T00:00:00Z', '2013-01-01T00:00:00Z')), (-0.21794406453770337, ('Southend United F.C.', '2008-01-01T00:00:00Z', '2008-01-01T00:00:00Z')), (-0.21794410924118696, ('Southend United F.C.', '2008-01-01T00:00:00Z', '2012-01-01T00:00:00Z')), (-0.3215548330781649, ('Wycombe Wanderers F.C.', '2006-01-01T00:00:00Z', '2007-01-01T00:00:00Z')), (-0.3403424853322695, ('Oldham Athletic A.F.C.', '2006-01-01T00:00:00Z', '2006-01-01T00:00:00Z')), (-0.3469381073343478, ('Luton Town F.C.', '2007-01-01T00:00:00Z', '2008-01-01T00:00:00Z')), (-0.7174502702528718, ('Chelsea F.C.', '2004-01-01T00:00:00Z', '2008-01-01T00:00:00Z'))]
Topk Contexts: [('Port Vale F.C.', '2015-01-01T00:00:00Z', 'unknown'), ('Crewe Alexandra F.C.', '2013-01-01T00:00:00Z', '2015-01-01T00:00:00Z'), ('S

C:\Users\Win10\AppData\Local\Temp\ipykernel_2308\3988018708.py:3: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  qt = np.datetime64(query_time)
C:\Users\Win10\AppData\Local\Temp\ipykernel_2308\3988018708.py:4: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  dt = np.datetime64(doc_time)
C:\Users\Win10\AppData\Local\Temp\ipykernel_2308\1794962655.py:5: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  filtered_docs = [doc for score, doc in ranked_docs if np.datetime64(doc[1]) <= np.datetime64(query_time)]


Ranked Docs: [(2.0029018913422023, ('Secretary of National Security and Defense Council of Ukraine', '2014-12-16T00:00:00Z', '2019-05-19T00:00:00Z')), (1.7917647923199556, ("People's Deputy of Ukraine", '2014-11-27T00:00:00Z', '2015-01-14T00:00:00Z')), (1.5271248594775342, ('Chairman of the Verkhovna Rada', '2014-02-22T00:00:00Z', '2014-11-27T00:00:00Z')), (1.1653859094634198, ('Prime Minister of Ukraine', '2014-02-22T00:00:00Z', '2014-02-27T00:00:00Z')), (1.1376605319162896, ('President of Ukraine', '2014-02-23T00:00:00Z', '2014-06-07T00:00:00Z')), (0.7802784084044612, ("People's Deputy of Ukraine", '2012-12-12T00:00:00Z', '2014-11-27T00:00:00Z')), (-0.11631392511937316, ('Prime Minister of Ukraine', '2010-03-04T00:00:00Z', '2010-03-11T00:00:00Z')), (-0.319103113198697, ("People's Deputy of Ukraine", '2007-11-23T00:00:00Z', '2007-12-19T00:00:00Z')), (-0.35211798862419785, ('First Deputy Prime Minister of Ukraine', '2007-12-01T00:00:00Z', '2010-11-03T00:00:00Z')), (-0.4880157309529717,

...

Ranked Docs: [(1.5632217500622037, ('Minister of Defense (Russia)', '2012-11-06T00:00:00Z', '2024-05-12T00:00:00Z')), (1.1816655641394567, ('governor of Moscow Oblast', '2012-05-11T00:00:00Z', '2012-11-06T00:00:00Z')), (-0.32825430673795397, ('Leader of United Russia party', '2001-12-01T00:00:00Z', '2005-04-15T00:00:00Z')), (-0.9912855749695428, ('Minister of Emergency Situations', '1991-04-17T00:00:00Z', '2012-05-11T00:00:00Z'))]
Topk Contexts: [('Minister of Defense (Russia)', '2012-11-06T00:00:00Z', '2024-05-12T00:00:00Z'), ('governor of Moscow Oblast', '2012-05-11T00:00:00Z', '2012-11-06T00:00:00Z'), ('Leader of United Russia party', '2001-12-01T00:00:00Z', '2005-04-15T00:00:00Z'), ('Minister of Emergency Situations', '1991-04-17T00:00:00Z', '2012-05-11T00:00:00Z')]
Contexts: [('Mustapha Harun', '1967-05-12T00:00:00Z', '1975-11-01T00:00:00Z'), ('Salleh Said Keruak', '1994-12-27T00:00:00Z', '1996-05-28T00:00:00Z'), ('Peter Lo Sui Yin', '1965-01-01T00:00:00Z', '1967-05-12T00:00:00Z')

C:\Users\Win10\AppData\Local\Temp\ipykernel_2308\3988018708.py:3: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  qt = np.datetime64(query_time)
C:\Users\Win10\AppData\Local\Temp\ipykernel_2308\3988018708.py:4: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  dt = np.datetime64(doc_time)
C:\Users\Win10\AppData\Local\Temp\ipykernel_2308\1794962655.py:5: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  filtered_docs = [doc for score, doc in ranked_docs if np.datetime64(doc[1]) <= np.datetime64(query_time)]


Ranked Docs: [(4.084290435658394, ('Shafie Apdal', '2018-05-12T00:00:00Z', '2020-07-30T00:00:00Z')), (0.22054968471342526, ('Chong Kah Kiat', '2001-03-27T00:00:00Z', '2003-03-27T00:00:00Z')), (0.15050181116059957, ('Salleh Said Keruak', '1994-12-27T00:00:00Z', '1996-05-28T00:00:00Z')), (0.14342888615177823, ('Musa Aman', '2003-03-27T00:00:00Z', '2018-05-12T00:00:00Z')), (0.10956363284593293, ('Bernard Giluk Dompok', '1998-05-28T00:00:00Z', '1999-03-14T00:00:00Z')), (0.07482466467226717, ('Sakaran Dandai', '1994-03-17T00:00:00Z', '1994-12-27T00:00:00Z')), (0.06667181507501008, ('Yong Teck Lee', '1996-05-28T00:00:00Z', '1998-05-28T00:00:00Z')), (0.06522676988081899, ('Joseph Pairin Kitingan', '1985-04-22T00:00:00Z', '1994-03-17T00:00:00Z')), (0.04026032866992155, ('Osu Sukam', '1999-03-14T00:00:00Z', '2001-03-27T00:00:00Z')), (-0.02160116390748057, ('Mustapha Harun', '1985-04-22T00:00:00Z', '1985-04-22T00:00:00Z')), (-0.03969997464935554, ('Harris Salleh', '1976-06-06T00:00:00Z', '1985-0

C:\Users\Win10\AppData\Local\Temp\ipykernel_2308\3988018708.py:3: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  qt = np.datetime64(query_time)
C:\Users\Win10\AppData\Local\Temp\ipykernel_2308\3988018708.py:4: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  dt = np.datetime64(doc_time)
C:\Users\Win10\AppData\Local\Temp\ipykernel_2308\1794962655.py:5: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  filtered_docs = [doc for score, doc in ranked_docs if np.datetime64(doc[1]) <= np.datetime64(query_time)]


Question: List all sports teams Anthony Grant, also known as Anthony Paul Shaun Andrew Daure Grant, played for from 2010 to 2020.
Prediction: Southend United F.C. (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020); Crewe Alexandra FC (2013, 2014)

Question: List all positions Oleksandr Turchynov, also known as Oleksandr Valentynovych Turchynov, held from 2010 to 2020.
Prediction: Secretary of State for National Security and Defence (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020); Prime Minister of Ukraine (2020)

Question: List all political parties Paul Murphy was a member of from 2010 to 2020.
Prediction: Conservative Party (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all sports teams Andy Carroll, also known as Andrew Thomas Carroll, played for from 2010 to 2020.
Prediction: England national under-21 football team (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020); West Ham United F.C. (2019, 20

In [17]:
# Save predictions to a JSON file
with open('predictions-FlanT5-small-RAG-temporal.json', 'w') as f:
    json.dump(predictions, f)

## Evaluation

In [1]:
from datasets import load_dataset
import json

In [2]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
import re

# Define metric functions
def entity_match(predictions, references):
    scores = []
    for pred_list, ref_list in zip(predictions, references):
        pred_entities = set([re.sub(r'\s*\(.*?\)', '', pred.strip().lower()) for pred in pred_list])
        ref_entities = set([re.sub(r'\s*\(.*?\)', '', ref.strip().lower()) for ref in ref_list])

        # Debug prints
        # print("Pred Entities:", pred_entities)
        # print("Ref Entities:", ref_entities)

        matches = len(pred_entities & ref_entities)  # Intersection of sets
        scores.append(matches / len(ref_entities) if ref_entities else 0)
    return sum(scores) / len(scores)


def timeline_match(predictions, references):
    def extract_years(text):
        matches = re.findall(r'\d{4}', text)
        if matches:
            return set(map(int, matches))
        return set()

    scores = []
    for pred_list, ref_list in zip(predictions, references):
        match_scores = []
        for i, ref in enumerate(ref_list):
            ref_years = extract_years(ref)
            try:
                pred_years = extract_years(pred_list[i])
                # print(f"Matching prediction timeline {i + 1} with reference timeline {i + 1}")
                # print("Pred Years:", pred_years)
                # print("Ref Years:", ref_years)
                matches = len(pred_years & ref_years)
                match_scores.append(matches / len(ref_years) if ref_years else 0)
            except IndexError:
                # print(f"No prediction timeline for reference timeline {i + 1}")
                match_scores.append(0)
        scores.append(sum(match_scores) / len(match_scores) if match_scores else 0)
    return sum(scores) / len(scores)

def f1_metric(predictions, references):
    def compute_f1(pred_list, ref_list):
        pred_tokens = set(pred_list)
        ref_tokens = set(ref_list)

        if not ref_tokens:
            return 0.0

        true_positives = len(pred_tokens & ref_tokens)
        precision = true_positives / len(pred_tokens) if pred_tokens else 0
        recall = true_positives / len(ref_tokens) if ref_tokens else 0

        if precision + recall == 0:
            return 0.0

        return 2 * (precision * recall) / (precision + recall)

    f1_scores = []

    for pred_list, ref_list in zip(predictions, references):
        # Flatten the lists for token-level comparison
        all_preds = [token for pred in pred_list for token in pred.split()]
        all_refs = [token for ref in ref_list for token in ref.split()]
        f1_scores.append(compute_f1(all_preds, all_refs))

    return sum(f1_scores) / len(f1_scores) if f1_scores else 0

def extract_years(text):
    matches = re.findall(r'\d{4}', text)
    if matches:
        years = list(map(int, matches))
        return min(years), max(years)  # Return the earliest and latest years
    return None, None

def time_metric(predictions, references):
    time_diffs = []
    
    for pred_list, ref_list in zip(predictions, references):
        for pred, ref in zip(pred_list, ref_list):
            pred_start, pred_end = extract_years(pred)
            ref_start, ref_end = extract_years(ref)
            
            # print(f"Prediction: {pred}, Extracted Years: ({pred_start}, {pred_end})")  # Debugging
            # print(f"Reference: {ref}, Extracted Years: ({ref_start}, {ref_end})")  # Debugging
            
            if pred_start is not None and ref_start is not None:
                time_diff = abs(pred_start - ref_start) + abs(pred_end - ref_end)
                time_diffs.append(time_diff)
            else:
                time_diffs.append(float('inf'))  # Invalid prediction or reference

    valid_diffs = [diff for diff in time_diffs if diff != float('inf')]
    if not valid_diffs:
        return float('inf')  # If all are invalid, return inf
    return sum(valid_diffs) / len(valid_diffs) # Average time difference

def completeness(predictions, references):
    scores = []
    
    for pred_list, ref_list in zip(predictions, references):
        pred_items = set([item.strip().lower() for pred in pred_list for item in pred.split(", ")])
        ref_items = set([item.strip().lower() for ref in ref_list for item in ref.split(", ")])
        
        correct_count = len(pred_items.intersection(ref_items))
        total_count = len(ref_items)
        
        scores.append(correct_count / total_count if total_count > 0 else 0)
    return sum(scores) / len(scores)

In [3]:
# Load the test dataset
test_dataset = load_dataset('json', data_files='./data/test_TLQA.json')['train']

In [4]:
# References
references = [ans for ans in test_dataset['answers']]  # Ensure references are lists of answers

In [5]:
# Load predictions from a JSON file
with open('predictions-FlanT5-small-RAG-temporal.json', 'r') as f:
    predictions = json.load(f)

In [6]:
# Convert predictions to the required list format
predictions = [pred.split("; ") for pred in predictions]

In [7]:
# Debug: Print some predictions and references to ensure alignment
for pred, ref in zip(predictions, references):
    print("Prediction:", pred)
    print("Reference:", ref)

Prediction: ['Southend United F.C. (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)', 'Crewe Alexandra FC (2013, 2014)']
Reference: ['Southend United F.C. (2010, 2011, 2012)', 'Stevenage F.C. (2012, 2013)', 'Crewe Alexandra F.C. (2013, 2014, 2015)', 'Port Vale F.C. (2015, 2016, 2017, 2018, 2019, 2020)']
Prediction: ['Secretary of State for National Security and Defence (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)', 'Prime Minister of Ukraine (2020)']
Reference: ['Prime Minister of Ukraine (2010)', 'First Deputy Prime Minister of Ukraine (2010)', "People's Deputy of Ukraine (2012, 2013, 2014, 2015)", 'Chairman of the Verkhovna Rada (2014, 2015, 2016, 2017, 2018, 2019, 2020)', 'President of Ukraine (2014)']
Prediction: ['Conservative Party (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)']
Reference: ['Socialist Party (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019)', 'RISE (Ireland) (2019, 2020)']
Prediction: ['Englan

In [8]:
# Evaluate predictions with custom metrics
entity_score = entity_match(predictions, references)
timeline_score = timeline_match(predictions, references)
f1 = f1_metric(predictions, references)
time_metric_score = time_metric(predictions, references)
completeness_score = completeness(predictions, references)

print(f"Entity Match: {entity_score}")
print(f"Timeline Match: {timeline_score}")
print(f"F1 Score: {f1}")
print(f"Time Metric: {time_metric_score}")
print(f"Completeness: {completeness_score}")

Entity Match: 0.28817271016150575
Timeline Match: 0.4486695719252902
F1 Score: 0.6130569717532062
Time Metric: 6.1052307692307695
Completeness: 0.5625521262255604
